<hr></hr>

# XY Belt Calibration

The OT1 uses elastic bands to travel along its X (left-right) and Y (forward-backward) axis. Very slight changes in length from stretching will cause the robot to slowly lose its ability to measure distances accurately. For example, we tell the robot to move 100mm, but it seems to move about 103mm.

There is a solution! What we can do is simply tell the robot to move a certain distance, then measure and tell it how far it actually travelled. If done accurately, this will then fix it's internal calculations for determining "real-world" distances.

<div class="alert alert-warning" role="alert" style="margin: 10px">
<p><strong>NOTE</strong></p>
<p>Be very careful when changing the XY belt calibrations, as innacurate settings could make the robot move uncontrollably. If things go wrong and you wish to start fresh, you can always <a href="http://docs.opentrons.com/updating_firmware.html">reload the firmware</a>.</p>
</div>

<div class="alert alert-warning" role="alert" style="margin: 10px">
<p><strong>NOTE</strong></p>
<p>Distance accuracy is different from consistency. If your robot cannot move to the same location consistently, this is a different problem and more to do with the tension of your belts. Please contact Opentrons support to help troubleshoot solutions.</p>
</div>

In [ ]:
from opentrons import robot

ports = robot.get_serial_ports_list()
for p in robot.get_serial_ports_list():
    print(p)

# TO CONNECT, UNCOMMENT THE BELOW LINE AND EDIT WITH THE NAME OF YOUR PORT
robot.connect('/dev/tty.usbmodem14141')

if not robot.is_connected():
    raise RuntimeError('No connection to robot')

print('Connected, homing...')

robot.home()
robot.home('xy')

print('Moving the robot {0}mm on the X, and {1}mm on the Y'.format(dimensions[0], dimensions[1]))

dimensions = robot._driver.get_dimensions() - (50, 50, 0)
robot.move_head(x=dimensions[0], y=-dimensions[1], mode='relative')

input('mark X position on the robot, then press ENTER key')
input('mark Y position on the robot, then press ENTER key')

robot.home('xy')
robot.home('xy')

actual_travel_x = float(input('enter the measured X distance: '))
actual_travel_y = float(input('enter the measured Y distance: '))

robot._driver.calibrate_steps_per_mm('x', dimensions[0], actual_travel_x)
robot._driver.calibrate_steps_per_mm('y', dimensions[1], actual_travel_y)

print('New values saved. Restarting the robot to allow changes to take effect')
print('Please wait 5-10 seconds before reconnecting to the robot')

robot._driver.reset()
robot.disconnect()